# SageMaker를 사용한 ML 모델 등록
ML 모델을 훈련한 후, 프로덕션 환경에서 사용하기 전에 조직 내 데이터 과학자나 MLOps 엔지니어가 모델의 성능을 평가하고 검토하기를 원할 수 있습니다. 이를 위해 SageMaker 모델 레지스트리에 모델 버전을 등록할 수 있습니다. SageMaker 모델 레지스트리는 데이터 과학자나 엔지니어가 머신 러닝(ML) 모델을 카탈로그화하고 모델 버전과 관련 메타데이터(예: 훈련 지표)를 관리하는 데 사용할 수 있는 저장소입니다. 또한 모델의 승인 상태를 관리하고 기록할 수도 있습니다.

SageMaker 모델 레지스트리에 모델 버전을 등록한 후, 데이터 과학자나 MLOps 팀은 SageMaker Studio를 통해 SageMaker 모델 레지스트리에 접근할 수 있습니다. 또한 데이터 과학자나 MLOps 팀은 모델을 평가하고 승인 상태를 업데이트할 수 있습니다. 모델이 요구 사항을 충족하지 못하면 데이터 과학자나 MLOps 팀은 상태를 `Rejected`로 업데이트할 수 있습니다. 모델이 요구 사항을 충족하면 데이터 과학자나 MLOps 팀은 상태를 `Approved`로 업데이트할 수 있습니다. 그런 다음 모델을 엔드포인트에 배포하거나 CI/CD 파이프라인으로 모델 배포를 자동화할 수 있습니다.

SageMaker 모델 레지스트리 기능을 사용하여 모델을 조직의 MLOps 프로세스와 원활하게 통합할 수 있습니다.

다음 다이어그램은 MLOps 워크플로우에 통합하기 위해 SageMaker Studio에서 구축된 모델 버전을 SageMaker 모델 레지스트리에 등록하는 예를 요약한 것입니다.

![register model](img/sagemaker-mlops-register-model-diagram.jpg)

## 모델 등록
모델 레지스트리를 사용하면 소프트웨어 코드의 버전 관리와 유사하게 훈련한 모델의 버전을 관리할 수 있습니다. 버전 관리가 활성화되면 시간이 지남에 따라 모델의 성능을 추적하고 프로덕션 환경에서 사용할 최상의 모델에 대해 정보에 기반한 결정을 내릴 수 있습니다. SageMaker 모델 레지스트리는 각 그룹에 모델 패키지가 있는 여러 모델(패키지) 그룹으로 구성됩니다. 이러한 모델 그룹은 선택적으로 하나 이상의 컬렉션에 추가될 수 있습니다. 모델 그룹의 각 모델 패키지는 훈련된 모델에 해당합니다. 각 모델 패키지의 버전은 1부터 시작하여 모델 그룹에 새 모델 패키지가 추가될 때마다 증가하는 숫자 값입니다. 예를 들어, 모델 그룹에 5개의 모델 패키지가 추가되면 모델 패키지 버전은 1, 2, 3, 4, 5가 됩니다.

다음 다이어그램은 SageMaker 모델 레지스트리에서 모델 버전 관리가 어떻게 구성되는지 보여줍니다:

![sm model registry](img/sagemaker-model-registry-diagram.jpg)

In [ ]:
%pip install sagemaker mlflow==2.13.2 sagemaker-mlflow==0.1.0

관련 라이브러리 가져오기

In [ ]:
import json
import os
import sagemaker
import boto3
import mlflow
from time import gmtime, strftime
from sagemaker.model import Model
from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
    FileSource
)
from sagemaker import Model
from sagemaker.model_card.model_card import ModelCard, TrainingDetails, TrainingJobDetails, ModelOverview
from botocore.exceptions import ClientError

# 헬퍼 함수 정의

In [ ]:
def download_from_s3(s3_client, local_file_path, bucket_name, s3_file_path):
    try:
        # Download the file
        s3_client.download_file(bucket_name, s3_file_path, local_file_path)
        print(f"File downloaded successfully to {local_file_path}")
        return True
    except ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            print(f"An error occurred: {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

def upload_to_s3(s3_client, local_file_path, bucket_name, s3_file_path=None):
    # If S3 file path is not specified, use the basename of the local file
    if s3_file_path is None:
        s3_file_path = os.path.basename(local_file_path)

    try:
        # Upload the file
        s3_client.upload_file(local_file_path, bucket_name, s3_file_path)
        print(f"File {local_file_path} uploaded successfully to {bucket_name}/{s3_file_path}")
        return True
    except ClientError as e:
        print(f"ClientError: {e}")
        return False
    except FileNotFoundError:
        print(f"The file {local_file_path} was not found")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False
        
def write_params(s3_client, step_name, params, notebook_param_s3_bucket_prefix):
    local_file_path = f"{step_name}.json"
    with open(local_file_path, "w") as f:
        f.write(json.dumps(params))
    base_local_file_path = os.path.basename(local_file_path)
    bucket_name = notebook_param_s3_bucket_prefix.split("/")[2] # Format: s3://<bucket_name>/..
    s3_file_path = os.path.join("/".join(notebook_param_s3_bucket_prefix.split("/")[3:]), base_local_file_path)
    upload_to_s3(s3_client, local_file_path, bucket_name, s3_file_path)
    
def read_params(s3_client, notebook_param_s3_bucket_prefix, step_name):
    local_file_path = f"{step_name}.json"
    base_local_file_path = os.path.basename(local_file_path)
    bucket_name = notebook_param_s3_bucket_prefix.split("/")[2] # Format: s3://<bucket_name>/..
    s3_file_path = os.path.join("/".join(notebook_param_s3_bucket_prefix.split("/")[3:]),  base_local_file_path)
    downloaded = download_from_s3(s3_client, local_file_path, bucket_name, s3_file_path)
    with open(local_file_path, "r") as f:
        data = f.read()
        params = json.loads(data)
    return params


# 변수 초기화
이 저장소의 이전 노트북과 마찬가지로, 다음 셀에 정의된 변수는 이 노트북 전체에서 특별히 사용됩니다. 하드코딩된 값 외에도, 이러한 변수는 노트북이 SageMaker Pipeline 작업이나 SageMaker Project를 통한 CICD 파이프라인과 같이 원격으로 실행되도록 예약될 때 매개변수로 노트북에 전달될 수 있습니다. 다음 실습에서 이 노트북에 매개변수를 전달하는 방법에 대해 자세히 알아보겠습니다. 노트북 매개변수화에 대한 자세한 정보는 [이 문서](https://docs.aws.amazon.com/sagemaker/latest/dg/notebook-auto-run-troubleshoot-override.html)를 참조하세요.

`02-preprocess.ipynb` 노트북과 마찬가지로, 다음 변수는 SageMaker Studio 런처를 통해 얻을 수 있습니다. 추가 도움이 필요한 경우 노트북에 안내와 스크린샷이 제공됩니다.

In [ ]:
region = "us-east-1"
os.environ["AWS_DEFAULT_REGION"] = region
boto_session = boto3.Session(region_name=region)
sess = sagemaker.Session(boto_session=boto_session)
bucket_name = sess.default_bucket()
bucket_prefix = "player-churn/xgboost"
notebook_param_s3_bucket_prefix=f"s3://{bucket_name}/{bucket_prefix}/params"
experiment_name = "player-churn-model-experiment"
run_id = None
model_package_group_name = "player-churn-model-group" # Provide a new model package group name. For example: player-churn-model-group
mlflow_tracking_server_arn = "" # Provide a valid mlflow tracking server ARN. You can find the value in the output from 00-start-here.ipynb
model_approval_status = "PendingManualApproval"
model_statistics_s3_path = None
model_constraints_s3_path = None
model_data_statistics_s3_path = None
model_data_constraints_s3_path = None

In [ ]:
assert len(model_package_group_name) > 0
assert len(mlflow_tracking_server_arn) > 0

이전 노트북에서 단계 변수를 검색합니다.

In [ ]:
preprocess_step_name = "02-preprocess"
train_step_name = "03-train"
evaluation_step_name = "04-evaluation"

s3_client = boto3.client("s3", region_name=region)
preprocess_step_params = read_params(s3_client, notebook_param_s3_bucket_prefix, preprocess_step_name)
train_step_params = read_params(s3_client, notebook_param_s3_bucket_prefix, train_step_name)
evaluation_step_params = read_params(s3_client, notebook_param_s3_bucket_prefix, evaluation_step_name)
experiment_name = preprocess_step_params["experiment_name"]

다음 셀은 MLFlow 추적 서버를 이 모델 레지스트리 작업과 통합합니다.

In [ ]:
suffix = strftime('%d-%H-%M-%S', gmtime())
mlflow.set_tracking_uri(mlflow_tracking_server_arn)
experiment = mlflow.set_experiment(experiment_name=experiment_name)
run = mlflow.start_run(run_id=run_id) if run_id else mlflow.start_run(run_name=f"register-{suffix}", nested=True)

`04-evaluation` 단계에서 모델 평가 지표를 다운로드합니다. 이 지표는 SageMaker 모델 레지스트리에 등록된 특정 모델의 성능 특성을 제공합니다.

In [ ]:
local_file_path = "evaluation.json"
evaluation_result_s3_path = evaluation_step_params["evaluation_result_s3_path"]
s3_file_path = "/".join(evaluation_result_s3_path.split("/")[3:])
evaluation_result_bucket_name = evaluation_result_s3_path.split("/")[2]

In [ ]:
download_from_s3(s3_client, local_file_path, bucket_name, s3_file_path)

In [ ]:
mlflow.log_artifact(local_path=local_file_path)

사용 가능한 경우 모델 기준 지표를 캡처합니다.

In [ ]:
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=model_statistics_s3_path,
        content_type="application/json",
    ) if model_statistics_s3_path else None,
    model_constraints=MetricsSource(
        s3_uri=model_constraints_s3_path,
        content_type="application/json",
    ) if model_constraints_s3_path else None,
    model_data_statistics=MetricsSource(
        s3_uri=model_data_statistics_s3_path,
        content_type="application/json",
    ) if model_data_statistics_s3_path else None,
    model_data_constraints=MetricsSource(
        s3_uri=model_data_constraints_s3_path,
        content_type="application/json",
    ) if model_data_constraints_s3_path else None,
)



훈련 작업의 모델 정보를 사용하여 모델 등록 프로세스를 위해 SageMaker 훈련 작업에서 모델 아티팩트 세부 정보를 수집할 수 있습니다.

In [ ]:
XGBOOST_IMAGE_URI = sagemaker.image_uris.retrieve(
            "xgboost",
            region=boto3.Session().region_name,
            version="1.7-1"
)
model_data = train_step_params["model_s3_path"]
model_role = sagemaker.get_execution_role()
model_name = "player-churn-model"

In [ ]:
model = Model(image_uri=XGBOOST_IMAGE_URI, sagemaker_session=sess, model_data=model_data, role=model_role, name=model_name)
training_details = TrainingDetails.from_model_s3_artifacts(
                model_artifacts=[model_data], sagemaker_session=sess
            )
training_job_details = training_details.training_job_details
training_datasets = [preprocess_step_params["train_data"], preprocess_step_params["validation_data"]]
training_job_details.training_datasets = training_datasets

model_card = ModelCard(
    name="estimator_card",
    training_details=training_details,
    sagemaker_session=sess,
)

model_overview = ModelOverview(model_artifact=[model_data])
model_card.model_overview = model_overview

## SageMaker 모델 객체를 사용하여 모델 패키지 그룹 등록

In [ ]:
model_package = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.m5.xlarge", "ml.m5.large"],
    transform_instances=["ml.m5.xlarge", "ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
    domain="MACHINE_LEARNING",
    task="CLASSIFICATION",
    model_card=model_card
)

# SageMaker 모델 패키지 그룹
새로 등록된 모델을 시각화하려면 SageMaker Studio 런처로 이동하여 왼쪽 창에서 `Models`를 선택하면 오른쪽 창에 모델 그룹이 표시됩니다. 다음 다이어그램은 SageMaker Studio 콘솔에서 주어진 모델 패키지 그룹에 생성된 XGBoost 모델의 새 버전을 보여줍니다.

![sagemaker model registry](img/sagamaker-model-registry-diagram.jpg)

In [ ]:
mlflow.log_params({
    "model_package_arn":model_package.model_package_arn,
    "model_statistics_uri":model_statistics_s3_path if model_statistics_s3_path else '',
    "model_constraints_uri":model_constraints_s3_path if model_constraints_s3_path else '',
    "data_statistics_uri":model_data_statistics_s3_path if model_data_statistics_s3_path else '',
    "data_constraints_uri":model_data_constraints_s3_path if model_data_constraints_s3_path else '',
})